In [48]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy
import xarray as xr

from tqdm import tqdm

import os
import torch
from torch import nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [28]:
ds_sst = xr.open_dataset("../reanalysis_data/hadisst.1870-01-01_2018-12-01.nc")
ds_sss = xr.open_dataset("../reanalysis_data/salt.1980.nc")

In [47]:
NAtl_sst = ds_sst['sst'][:,110:141,89:201]

In [88]:
ds_sss = np.zeros((40,12,91,112))

iyr = 0
for filename in tqdm(os.listdir('../reanalysis_data/salt_raw')):
    ds_f = xr.open_dataset("../reanalysis_data/salt_raw/"+filename)
    ds_f.coords['lon'] = (ds_f.coords['lon'] + 180) % 360 - 180
    ds_f = ds_f.sortby(ds_f.lon)
    
    ds_sss[iyr,:,:,:] = (1000*ds_f['salt'][:,0,284:375,89:201])
    
    iyr += 1

100%|██████████| 40/40 [02:07<00:00,  3.19s/it]


In [91]:
ds_sss_monthly = ds_sss.reshape(40*12,91,112)

In [93]:
ds_sss_monthly.shape

(480, 91, 112)

DatetimeIndex(['1980-01-31', '1980-02-29', '1980-03-31', '1980-04-30',
               '1980-05-31', '1980-06-30', '1980-07-31', '1980-08-31',
               '1980-09-30', '1980-10-31',
               ...
               '2019-03-31', '2019-04-30', '2019-05-31', '2019-06-30',
               '2019-07-31', '2019-08-31', '2019-09-30', '2019-10-31',
               '2019-11-30', '2019-12-31'],
              dtype='datetime64[ns]', length=480, freq='M')

In [100]:
NAtl_sss = xr.DataArray(ds_sss_monthly, coords=[pd.date_range("1980-01-01", '2019-12-31',freq='M'),
                                     ds_f['salt'][:,0,284:375,89:201].lat, 
                                     ds_f['salt'][:,0,284:375,89:201].lon], dims=["time","lat", "lon"])



In [116]:
NAtl_sss.to_netcdf('../reanalysis_data/NAtl_sss.nc')

In [115]:
NAtl_sst.to_netcdf('../reanalysis_data/NAtl_sst.nc')